In [1]:
#Load pdf Files
from langchain_community.document_loaders import (
    PyPDFLoader,
    PyMuPDFLoader
)

c:\Users\ASUS\Desktop\RAGUdemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print("Pypdf Loader")
try:
    pypdfloader=PyPDFLoader(r"data\pdf\attention.pdf")
    docs=pypdfloader.load()
    print(f"Number of pages: {len(docs)}")
    print(f"First page: {docs[0].page_content[:100]}")
    print(f"First page metadata: {docs[0].metadata}")
except Exception as e:
    print(f"Error: {e}")

Pypdf Loader
Number of pages: 15
First page: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and
First page metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data\\pdf\\attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}


In [9]:
print("PyMupdfloader")
try:
    pymudpdf_loader=PyMuPDFLoader("data/pdf/attention.pdf",
                                  extract_images=True)
    docs=pymudpdf_loader.load()
    print(f"Number of pages: {len(docs)}")
    print(f"First page: {docs[0].page_content[:100]}")
    print(f"First page metadata: {docs[0].metadata}")
except Exception as e:
    print(f"error:{e}")


PyMupdfloader
Number of pages: 15
First page: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and
First page metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': 'data/pdf/attention.pdf', 'file_path': 'data/pdf/attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}
